In [1]:
import rasterio
import os
from shapely.geometry import box
from rasterio.windows import from_bounds, bounds
import matplotlib.pyplot as plt
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [36]:
path_slope = 'Slope_SRTM_Zone_WGS84.tif'
path_pp1 = 'output/2024_09_11.tif'
path_output = 'output/'
with rasterio.open(path_slope) as src1, rasterio.open(path_pp1) as src2:
    ext1 = box(*src1.bounds)
    ext2 = box(*src2.bounds)
    print(f'ext1: {ext1}\next2: {ext2}')
    # intersection devuelve la geometria que se comparte entre las geometrias de entradas
    intersection = ext1.intersection(ext2)
    print(f'intersection: {intersection}')
    
    window = from_bounds(*intersection.bounds, src1.transform)
    print(f'Ventanas: {window}')
    data = src1.read(window=window)
    print(f'Tipo de dato {type(data)}\nDatos: {data}')
    print('bounds:', bounds(window, src1.transform))
    left, bottom, right, top = bounds(window, src1.transform)
    #calculate default transform: (source coor, target coor, width, height, left, bot,right, top)
    #print('src1.crs',src1.crs)
    #print('src2.crs',src2.crs)
    print('data.shape[1]', data.shape[1])
    print('data.shape[2]', data.shape[2])
    new_transform, width, height = calculate_default_transform(src1.crs, src1.crs, data.shape[2], data.shape[1],
                                                               left, bottom, right, top)
    height = window.height
    width = window.width
    #new_transform, width, height = calculate_default_transform(src1.crs, src1.crs, width, height,
    #                                                           left, bottom, right, top)
    print(f'La ventana es: {width} X {height}, Teniendo: {window}')
    kwargs = src1.meta.copy()
    print(kwargs)
    kwargs.update({
        'width': width,
        'height': height
    })
    print(f'kwargs: {kwargs}')
    print(window)
    # Creación del nuevo path
    with rasterio.open(path_output + 'output.tif', 'w+', **kwargs) as dst:
        print('------------------------------------------\n',data)
        reproject(
            source=data,
            destination=rasterio.band(dst, 1),
            src_transform=src1.window_transform(window),
            src_crs=src1.crs,
            dst_crs=src2.crs,
            dst_transform=new_transform,
            resampling=Resampling.bilinear
        )
    
    
    """
    Pregunta:
    que hago
    como lo hago
    por que lo hago
    
    a partir de cual array obtengo los datos de los tif con los datos de las features
    """

ext1: POLYGON ((-70.82414754 -41.065264796, -70.82414754 -37.936163569, -74.042230979 -37.936163569, -74.042230979 -41.065264796, -70.82414754 -41.065264796))
ext2: POLYGON ((-73.09109941222857 -56.99615358715215, -73.09109941222857 -36.313880920410156, -78.54290771484375 -36.313880920410156, -78.54290771484375 -56.99615358715215, -73.09109941222857 -56.99615358715215))
intersection: POLYGON ((-74.042230979 -41.065264796, -74.042230979 -37.936163569, -73.09109941222857 -37.936163569, -73.09109941222857 -41.065264796, -74.042230979 -41.065264796))
Ventanas: Window(col_off=0.0, row_off=0.0, width=3426.112880852015, height=11262.0)
Tipo de dato <class 'numpy.ndarray'>
Datos: [[[-9999. -9999. -9999. ... -9999. -9999. -9999.]
  [-9999. -9999. -9999. ... -9999. -9999. -9999.]
  [-9999. -9999. -9999. ... -9999. -9999. -9999.]
  ...
  [-9999. -9999. -9999. ... -9999. -9999. -9999.]
  [-9999. -9999. -9999. ... -9999. -9999. -9999.]
  [-9999. -9999. -9999. ... -9999. -9999. -9999.]]]
bounds: (-7

In [42]:
import rasterio
from rasterio.windows import Window

output_tif_path = 'output/output.tif'

with rasterio.open(output_tif_path) as src:
    meta = src.meta
    print('Metadatos: ',meta)
    data = src.read(1)
    print('Datos: ',data)
    width = meta['width']
    height = meta['height']
    print(f'Width: {width}\nHeight: {height}')
    window = src.window(*src.bounds)
    print('Ventana: ',window)
    data_window = src.read(1, window=window)
    print('Datos de la ventana: ',data_window)
    
    #lon, lat = -73.219497,-37.7517708
    lon,lat = -73.5688,-40.2737
    row, col = src.index(lon, lat)
    print(f'Row: {row}\nCol: {col}')
    value = data_window[row,col]
    print(f'Valor en ({lon},{lat}): {value}')
    
    

Metadatos:  {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -9999.0, 'width': 3426, 'height': 11262, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0002776124429779156, 0.0, -74.042230979,
       0.0, -0.0002778459622624755, -37.936163569)}
Datos:  [[-9999. -9999. -9999. ... -9999. -9999. -9999.]
 [-9999. -9999. -9999. ... -9999. -9999. -9999.]
 [-9999. -9999. -9999. ... -9999. -9999. -9999.]
 ...
 [-9999. -9999. -9999. ... -9999. -9999. -9999.]
 [-9999. -9999. -9999. ... -9999. -9999. -9999.]
 [-9999. -9999. -9999. ... -9999. -9999. -9999.]]
Width: 3426
Height: 11262
Ventana:  Window(col_off=0.0, row_off=0.0, width=3426.0, height=11262.0)
Datos de la ventana:  [[-9999. -9999. -9999. ... -9999. -9999. -9999.]
 [-9999. -9999. -9999. ... -9999. -9999. -9999.]
 [-9999. -9999. -9999. ... -9999. -9999. -9999.]
 ...
 [-9999. -9999. -9999. ... -9999. -9999. -9999.]
 [-9999. -9999. -9999. ... -9999. -9999. -9999.]
 [-9999. -9999. -9999. ... -9999. -9999. -9999.]]
Row: 8413
Col

In [160]:
import numpy as np

from osgeo import gdal, gdalconst, osr
from netCDF4 import Dataset

In [148]:
# convertir archivo nc a tif
nc_file_path = 'WRFProno_20240911061623.nc'
nc_file = Dataset(nc_file_path, 'r')

lon = nc_file.variables['XLONG'][:]
lat = nc_file.variables['XLAT'][:]
lon_min, lon_max = lon.min(), lon.max()
lat_min, lat_max = lat.min(), lat.max()
print(f'Latitud {lat_min} {lat_max}')
print(f'Longitud {lon_min} {lon_max}')

cols, rows = len(lon[0]), len(lat)
dx = (lon_max - lon_min) / cols
dy = (lat_max - lat_min) / rows
transform = [lon_min, dx, 0, lat_max, 0, -dy]

rainnc = nc_file.variables['RAINNC']
string_date = []

for time_index, timestamp_bytes in enumerate(nc_file.variables['Times']):
    timestamp_str = b''.join(timestamp_bytes).decode('utf-8')
    string_date.append(timestamp_str[:10])

fec_uni = set(string_date)
print(fec_uni)
for ind in fec_uni:
    ind_fec = [indice for indice, valor in enumerate(string_date) if valor == ind]
    print(max(ind_fec), min(ind_fec))
    if min(ind_fec) > 0:
        data1 = rainnc[max(ind_fec), :, :] - rainnc[min(ind_fec)-1, :, :]
    else:
        data1 = rainnc[max(ind_fec), :, :]
    print(data1.shape)
    
    output_tif_path = fr'output/{ind[:4]}_{ind[5:7]}_{ind[8:10]}.tif'    
    driver = gdal.GetDriverByName('GTiff')
    output_tiff = driver.Create(output_tif_path, rows, cols, 1, gdalconst.GDT_Float32)

    output_tiff.SetGeoTransform(transform)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)
    output_tiff.SetProjection(srs.ExportToWkt())
    
    data = np.flipud(data1) # No entiendo
    #data = data1
    output_tiff.GetRasterBand(1).WriteArray(data)
    output_tiff = None
nc_file.close()
    




    


Latitud -48.062965393066406 -36.313880920410156
Longitud -78.54290771484375 -68.9459228515625
{'2024-09-15', '2024-09-13', '2024-09-12', '2024-09-14', '2024-09-16', '2024-09-11'}
119 96
(213, 120)
71 48
(213, 120)
47 24
(213, 120)
95 72
(213, 120)
120 120
(213, 120)
23 0
(213, 120)
